# Load libraries and set device up

In [ ]:
import seaborn           as sns
import matplotlib.pyplot as plt
import torch
import json

import libraries.model   as clm
import libraries.dataset as cld

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sns.set_theme()

# Define parameters

In [ ]:
target = 'bandgap'

r_folder = f'models/GNN'
t_folder = f'data/bandgap-EPA/'
model_name = f'{r_folder}/{target}-model.pt'

# Generate or load graph database for training

Reference dataset

In [ ]:
r_dataset_name                     = f'{t_folder}/dataset.pt'
r_dataset_name_std                 = f'{t_folder}/standardized_dataset.pt'
r_standardized_parameters_name_std = f'{t_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
r_dataset     = torch.load(r_dataset_name)
r_dataset_std = torch.load(r_dataset_name_std)

# Load the data from the JSON file
with open(r_standardized_parameters_name_std, 'r') as json_file:
    numpy_dict = json.load(json_file)

# Convert torch tensors to numpy arrays
r_standardized_parameters = {}
for key, value in numpy_dict.items():
    try:
        r_standardized_parameters[key] = torch.tensor(value)
    except:
        r_standardized_parameters[key] = value

# Load the data from the JSON file
with open(f'{t_folder}/dataset_parameters.json', 'r') as json_file:
    r_dataset_parameters = json.load(json_file)

# Determine the index for the desired property
target_idx = [idx for idx, t in enumerate(r_dataset_parameters['target']) if t == target][0]
r_standardized_parameters['target_mean'] = r_standardized_parameters['target_mean'][target_idx]
r_standardized_parameters['target_std']  = r_standardized_parameters['target_std'][target_idx]

# Defining target factor
target_factor = r_standardized_parameters['target_std'] / r_standardized_parameters['scale']

# Select the target we want
ground_truths = [data.y[target_idx].item() for data in r_dataset]
del r_dataset

Prediciton dataset

In [ ]:
p_dataset_name = f'{r_folder}/dataset.pt'

# Load the standardized dataset, with corresponding labels and parameters
p_dataset = torch.load(p_dataset_name)

# Standardize the dataset with
p_dataset_std = cld.standarize_dataset_from_keys(p_dataset, r_standardized_parameters)

# Load model and make predictions

In [ ]:
# Load Graph Neural Network model (making room for temperature as node attribute)
model = clm.GCNN(features_channels=r_dataset[0].num_node_features+1, pdropout=0)

# Moving model to device
model = model.to(device)

# Load Graph Neural Network model
model.load_state_dict(torch.load(model_name, map_location=torch.device(device)))
model.eval()

# Compute predictions
predictions, uncertainties = clm.make_predictions(r_dataset_std, p_dataset, model, r_standardized_parameters)

# Plot predictions

In [ ]:
plt.errorbar(ground_truths, predictions, yerr=uncertainties,
             fmt='o', label='CLUE')

plt.xlabel(r'$T$ (K)')
plt.ylabel(r'$F_v$ (meV/atom)')
plt.legend(loc='best')
plt.savefig('UQ-comparison.pdf', dpi=50, bbox_inches='tight')
plt.show()